# Load Dataset

In [1]:
import pandas as pd
import numpy as np

In [2]:
class Loaddata(object):
    def __init__(self):
        folder = "C:/Users/Bin/Documents/Datasets/KDD99/"
        col_name_suffix = "columns.txt"
        dataset_suffix = "kddcup.data_10_percent_corrected"
        
    def read(self):
        if True:
            with open(folder+col_name_suffix) as col_file:
                line = col_file.readline()
            columns = line.split('.')
            col_names = []
            col_types = []
            for col in columns:
                col_names.append(col.split(': ')[0].strip())
                col_types.append(col.split(': ')[1])
            col_names.append("label")
            df = pd.read_csv(folder+dataset_suffix,names=col_names)
            data = df.iloc[:,np.array(pd.Series(col_types)=="continuous")].as_matrix()   #Select only numeric features
            label = df.iloc[:,-1]

            # Scaling
            scaler = MinMaxScaler()
            scaler.fit(data)
            dataset = scaler.transform(data) 
            
            return dataset
            
        else:
            print("Wrong dataset name")
            return 